## Mockdb 관련 스크립트

In [1]:
import os
from dotenv import load_dotenv

# 데이터베이스 관련
from sqlalchemy import create_engine, text

load_dotenv('.env')

# PostgreSQL 설정 로드
PG_HOST = os.getenv('PG_HOST')
PG_PORT = os.getenv('PG_PORT')
PG_DATABASE = os.getenv('PG_DATABASE')
PG_USER = os.getenv('PG_USER')
PG_PASSWORD = os.getenv('PG_PASSWORD')

print(f"PostgreSQL 연결 정보:")
print(f"   Host: {PG_HOST}")
print(f"   Port: {PG_PORT}")
print(f"   Database: {PG_DATABASE}")
print(f"   User: {PG_USER}")

# SQLAlchemy 연결 문자열 생성
POSTGRES_URL = f"postgresql://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DATABASE}"

print(f"\n✅ PostgreSQL 설정 로드 완료")

PostgreSQL 연결 정보:
   Host: dev-rubicon-postgresql.postgres.database.azure.com
   Port: 5432
   Database: postgres
   User: rubicon

✅ PostgreSQL 설정 로드 완료


In [2]:
def postgresql_connection_and_create_tables():
    """PostgreSQL 연결 및 테이블 생성"""
    
    print("=" * 60)
    print("PostgreSQL 연결 및 테이블 생성")
    print("=" * 60)
    
    try:
        # SQLAlchemy 엔진 생성
        engine = create_engine(POSTGRES_URL)
        
        # 연결 테스트
        with engine.connect() as conn:
            # 기본 연결 테스트
            result = conn.execute(text("SELECT version();"))
            version = result.fetchone()[0]
            print(f"✅ PostgreSQL 연결 성공!")
            print(f"   버전: {version}")
            
            # 현재 데이터베이스 정보
            result = conn.execute(text("SELECT current_database(), current_user;"))
            db_info = result.fetchone()
            print(f"   현재 DB: {db_info[0]}")
            print(f"   사용자: {db_info[1]}")
            
            existing_tables = [row[0] for row in result.fetchall()]
            print(f"\n📊 기존 테이블 수: {len(existing_tables)}개")
            if existing_tables:
                for table in existing_tables:
                    print(f"   • {table}")
        
        with engine.connect() as conn:
            # 트랜잭션 시작
            trans = conn.begin()
            
            try:
                # 1. 고객 정보 테이블
                conn.execute(text("""
                    CREATE TABLE IF NOT EXISTS sr_merged_product (
                        disp_lv1 VARCHAR(1000),
                        disp_lv2 VARCHAR(1000),
                        disp_lv3 VARCHAR(1000),
                        product_category_lv1 VARCHAR(1000),
                        product_category_lv2 VARCHAR(1000),
                        product_category_lv3 VARCHAR(1000),
                        model_name VARCHAR(1000),
                        mdl_code VARCHAR(1000),
                        goods_id VARCHAR(1000),
                        goods_nm VARCHAR(1000),
                        color VARCHAR(1000),
                        release_date DATE,
                        aisc_yn VARCHAR(1),
                        sc_yn VARCHAR(1),
                        gc_yn VARCHAR(1),
                        div_pay_apl_yn VARCHAR(1),
                        show_yn VARCHAR(1),
                        pd_url TEXT,
                        dlgt_img_url TEXT,
                        site_cd VARCHAR(10),
                        usp_desc VARCHAR,
                        review_num INT4,
                        estm_score NUMERIC(5,2),
                        sale_prc1 NUMERIC(10),
                        sale_prc2 NUMERIC(10),
                        sale_prc3 NUMERIC(10),
                        review_content TEXT,
                        disp_clsf_nm_list VARCHAR,
                        spec JSONB,
                        on_sale TEXT,
                        web_cd_dc_amt NUMERIC(10),
                        stock_qty INT4,
                        ctg_rank_recommend INT4,
                        ctg_rank_quantity INT4,
                        ctg_rank_rating INT4,
                        cstrt_mdl_codes VARCHAR,
                        card_promotion JSONB,
                        sale_prc NUMERIC(10)
                    );
                """))
                print("   ✅ sr_merged_product 테이블 생성")
                
                
                # 인덱스 생성
                conn.execute(text("CREATE INDEX IF NOT EXISTS idx_sr_merged_product_mdl_code ON sr_merged_product(mdl_code);"))
                conn.execute(text("CREATE INDEX IF NOT EXISTS idx_sr_merged_product_site_cd ON sr_merged_product(site_cd);"))
                conn.execute(text("CREATE INDEX IF NOT EXISTS idx_sr_merged_product_mdl_site ON sr_merged_product(mdl_code, site_cd);"))
                conn.execute(text("CREATE INDEX IF NOT EXISTS idx_sr_merged_product_goods_id ON sr_merged_product(goods_id);"))
                conn.execute(text("CREATE INDEX IF NOT EXISTS idx_sr_merged_product_goods_nm ON sr_merged_product(goods_nm);"))
                print("   ✅ 인덱스 생성 완료")


                # 커멘트 생성
                conn.execute(text("COMMENT ON TABLE sr_merged_product IS '상품 통합 정보 테이블';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.disp_lv1 IS '전시 대분류';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.disp_lv2 IS '전시 중분류';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.disp_lv3 IS '전시 소분류';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.product_category_lv1 IS '카테고리 대분류';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.product_category_lv2 IS '카테고리 중분류';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.product_category_lv3 IS '카테고리 소분류';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.model_name IS '모델 명(모델 코드 상위 집합)';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.mdl_code IS '모델 코드';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.goods_id IS '상품 아이디';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.goods_nm IS '상품 명';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.color IS '색상';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.release_date IS '출시일';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.aisc_yn IS 'ai구독 대상 여부';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.sc_yn IS 'ai구독 스마트 대상 여부';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.gc_yn IS '갤럭시 클럽 대상 여부';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.div_pay_apl_yn IS '나눠서 결제 대상 여부';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.show_yn IS '노출 여부';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.pd_url IS 'pd URL';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.dlgt_img_url IS '대표 이미지 URL';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.site_cd IS '채널코드';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.usp_desc IS '모델 카드 3줄 장점';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.review_num IS '해당 모델의 리뷰 개수';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.estm_score IS '해당 모델의 리뷰 점수';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.sale_prc1 IS '해당 모델의 기준가';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.sale_prc2 IS '해당 모델의 회원가';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.sale_prc3 IS '해당 모델의 혜택가';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.review_content IS '각 제품에 대한 리뷰들 모음';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.disp_clsf_nm_list IS '전시 분류명 모음';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.spec IS 'json 형태의 spec 모음';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.on_sale IS '판매여부';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.web_cd_dc_amt IS '웹 쿠폰 할인 가격';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.stock_qty IS '상품 재고';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.ctg_rank_recommend IS '전시 소분류내에서 우선 추천순 순위';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.ctg_rank_quantity IS '전시 소분류내에서 우선 판매량순 순위';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.ctg_rank_rating IS '전시 소분류내에서 우선 별점순 순위';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.cstrt_mdl_codes IS 'set상품의 경우 구성 제품 mdl_code';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.card_promotion IS 'card 할인 정보';"))
                conn.execute(text("COMMENT ON COLUMN sr_merged_product.sale_prc IS '최종가격';"))
                print("   ✅ 커멘트 생성 완료")

                # 트랜잭션 커밋
                trans.commit()
                print("\n✅ 모든 테이블 생성 완료!")
                
            except Exception as e:
                trans.rollback()
                print(f"❌ 테이블 생성 실패: {e}")
                return None
        
        return engine
        
    except Exception as e:
        print(f"❌ PostgreSQL 연결 실패: {e}")
        return None

# PostgreSQL 연결 및 테이블 생성
pg_engine = postgresql_connection_and_create_tables()

PostgreSQL 연결 및 테이블 생성
✅ PostgreSQL 연결 성공!
   버전: PostgreSQL 17.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 11.2.0, 64-bit
   현재 DB: postgres
   사용자: rubicon

📊 기존 테이블 수: 0개
   ✅ sr_merged_product 테이블 생성
   ✅ 인덱스 생성 완료
   ✅ 커멘트 생성 완료

✅ 모든 테이블 생성 완료!


In [3]:
# 로컬 파일의 데이터를 postgresql에 업로드

import pandas as pd
import json
from typing import List, Dict, Any
import re

import psycopg2
from psycopg2.extras import execute_batch
from datetime import datetime

from sqlalchemy import Engine, text

PG_UPLOAD_FILE_PATH = os.getenv('PG_UPLOAD_FILE_PATH')

def parse_json_field(value):
    """JSON 필드 파싱 헬퍼 함수"""
    if pd.isna(value) or value == '' or value is None:
        return None
        
    # 이미 dict인 경우
    if isinstance(value, dict):
        return json.dumps(value, ensure_ascii=False)
        
    # 문자열인 경우
    if isinstance(value, str):
        value = value.strip()
        if not value:
            return None
            
        try:
            # 중괄호로 시작하는 JSON 문자열
            if value.startswith('{'):
                # 먼저 그대로 파싱 시도
                try:
                    parsed = json.loads(value)
                    return json.dumps(parsed, ensure_ascii=False)
                except:
                    # 특수한 형식 처리 (예: {key: value} 형태)
                    # JSON 형식으로 변환 시도
                    value_cleaned = re.sub(r'(\w+):', r'"\1":', value)
                    value_cleaned = value_cleaned.replace('null', '"null"')
                    parsed = json.loads(value_cleaned)
                    return json.dumps(parsed, ensure_ascii=False)
            else:
                return None
        except Exception as e:
            print(f"JSON 파싱 실패: {value[:100]}... - {e}")
            return None
    
    return None

def prepare_data_from_csv(df: pd.DataFrame) -> List[tuple]:
    """
    CSV 데이터프레임을 PostgreSQL 삽입용 데이터로 변환
    
    Args:
        df: 원본 데이터프레임
        
    Returns:
        삽입용 튜플 리스트
    """
    # CSV 헤더와 테이블 컬럼 매핑
    column_mapping = {
        'disp_lv1': 'disp_lv1',
        'disp_lv2': 'disp_lv2',
        'disp_lv3': 'disp_lv3',
        'product_category_lv1': 'product_category_lv1',
        'product_category_lv2': 'product_category_lv2',
        'product_category_lv3': 'product_category_lv3',
        'model_name': 'model_name',
        'mdl_code': 'mdl_code',
        'goods_id': 'goods_id',
        'goods_nm': 'goods_nm',
        'color': 'color',
        'release_date': 'release_date',  
        'aisc_yn': 'aisc_yn',
        'sc_yn': 'sc_yn',
        'gc_yn': 'gc_yn',
        'div_pay_apl_yn': 'div_pay_apl_yn',
        'show_yn': 'show_yn',
        'pd_url': 'pd_url',
        'dlgt_img_url': 'dlgt_img_url',
        'site_cd': 'site_cd',
        'usp_desc': 'usp_desc',
        'review_num': 'review_num',
        'estm_score': 'estm_score',
        'sale_prc1': 'sale_prc1',
        'sale_prc2': 'sale_prc2',
        'sale_prc3': 'sale_prc3',
        'review_content': 'review_content',
        'disp_clsf_nm_list': 'disp_clsf_nm_list',
        'spec': 'spec',
        'on_sale': 'on_sale',
        'web_cp_dc_amt': 'web_cd_dc_amt',  # CSV의 web_cp_dc_amt -> 테이블의 web_cd_dc_amt
        'stock_qty': 'stock_qty',
        'ctg_rank_recommend': 'ctg_rank_recommend',
        'ctg_rank_quantity': 'ctg_rank_quantity',
        'ctg_rank_rating': 'ctg_rank_rating',
        'cstrt_mdl_codes': 'cstrt_mdl_codes',
        'card_promotion': 'card_promotion',
        'sale_prc': 'sale_prc'
    }
    
    prepared_data = []
    
    for idx, row in df.iterrows():
        try:
            record = []
            
            # 테이블 컬럼 순서대로 처리
            table_columns = [
                'disp_lv1', 'disp_lv2', 'disp_lv3',
                'product_category_lv1', 'product_category_lv2', 'product_category_lv3',
                'model_name', 'mdl_code', 'goods_id', 'goods_nm', 'color',
                'release_date', 'aisc_yn', 'sc_yn', 'gc_yn', 'div_pay_apl_yn',
                'show_yn', 'pd_url', 'dlgt_img_url', 'site_cd', 'usp_desc',
                'review_num', 'estm_score', 'sale_prc1', 'sale_prc2', 'sale_prc3',
                'review_content', 'disp_clsf_nm_list', 'spec', 'on_sale',
                'web_cd_dc_amt', 'stock_qty', 'ctg_rank_recommend',
                'ctg_rank_quantity', 'ctg_rank_rating', 'cstrt_mdl_codes',
                'card_promotion', 'sale_prc'
            ]
            
            # 역매핑 생성 (테이블 컬럼 -> CSV 컬럼)
            reverse_mapping = {v: k for k, v in column_mapping.items()}
            
            for table_col in table_columns:
                # CSV 컬럼명 찾기
                csv_col = reverse_mapping.get(table_col, table_col)
                
                # 데이터 가져오기
                if csv_col in df.columns:
                    value = row[csv_col]
                else:
                    value = None
                
                # NULL 처리
                if pd.isna(value) or value == '':
                    record.append(None)
                # DATE 처리 (release_date)
                elif table_col == 'release_date':
                    if value:
                        try:
                            # 다양한 날짜 형식 처리
                            date_val = pd.to_datetime(value)
                            record.append(date_val.date())
                        except Exception as e:
                            print(f"날짜 변환 실패 (행 {idx}): {value} - {e}")
                            record.append(None)
                    else:
                        record.append(None)
                # JSONB 처리 (spec, card_promotion)
                elif table_col in ['spec', 'card_promotion']:
                    json_value = parse_json_field(value)
                    record.append(json_value)
                # usp_desc 처리 (JSON 배열을 문자열로)
                elif table_col == 'usp_desc':
                    if value and isinstance(value, str) and value.startswith('{'):
                        # JSON 배열을 일반 문자열로 변환
                        try:
                            # 중괄호 제거하고 쉼표로 구분
                            cleaned = value.strip('{}')
                            # 따옴표 제거
                            cleaned = cleaned.replace('""', '"').replace('"', '')
                            record.append(cleaned)
                        except:
                            record.append(str(value) if value else None)
                    else:
                        record.append(str(value) if value else None)
                # NUMERIC 처리
                elif table_col in ['estm_score', 'sale_prc1', 'sale_prc2', 'sale_prc3', 'web_cd_dc_amt', 'sale_prc']:
                    if value is not None and value != '':
                        try:
                            record.append(float(value))
                        except:
                            record.append(None)
                    else:
                        record.append(None)
                # INT 처리
                elif table_col in ['review_num', 'stock_qty', 'ctg_rank_recommend', 'ctg_rank_quantity', 'ctg_rank_rating']:
                    if value is not None and value != '':
                        try:
                            record.append(int(float(value)))
                        except:
                            record.append(None)
                    else:
                        record.append(None)
                # VARCHAR(1) 처리 (Y/N 값들)
                elif table_col in ['aisc_yn', 'sc_yn', 'gc_yn', 'div_pay_apl_yn', 'show_yn']:
                    if value and str(value).upper() in ['Y', 'N']:
                        record.append(str(value).upper())
                    else:
                        record.append(None)
                # 나머지 VARCHAR/TEXT 처리
                else:
                    if value is not None and value != '':
                        record.append(str(value))
                    else:
                        record.append(None)
            
            prepared_data.append(tuple(record))
            
        except Exception as e:
            print(f"행 {idx} 처리 중 오류: {e}")
            continue
            
    print(f"총 {len(prepared_data)}개 레코드 준비 완료")
    return prepared_data

def insert_data_with_psycopg2(data: List[tuple], batch_size: int = 1000):
    """
    psycopg2를 직접 사용하여 데이터 삽입
    
    Args:
        data: 삽입할 데이터 튜플 리스트
        batch_size: 배치 크기
    """
    # psycopg2 직접 연결
    conn = psycopg2.connect(
        host=PG_HOST,
        port=PG_PORT,
        database=PG_DATABASE,
        user=PG_USER,
        password=PG_PASSWORD
    )
    
    insert_query = """
    INSERT INTO sr_merged_product (
        disp_lv1, disp_lv2, disp_lv3,
        product_category_lv1, product_category_lv2, product_category_lv3,
        model_name, mdl_code, goods_id, goods_nm, color,
        release_date, aisc_yn, sc_yn, gc_yn, div_pay_apl_yn,
        show_yn, pd_url, dlgt_img_url, site_cd, usp_desc,
        review_num, estm_score, sale_prc1, sale_prc2, sale_prc3,
        review_content, disp_clsf_nm_list, spec, on_sale,
        web_cd_dc_amt, stock_qty, ctg_rank_recommend,
        ctg_rank_quantity, ctg_rank_rating, cstrt_mdl_codes,
        card_promotion, sale_prc
    ) VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s
    ) ON CONFLICT DO NOTHING
    """
    
    try:
        cur = conn.cursor()
        
        total_records = len(data)
        inserted_count = 0
        failed_count = 0
        
        # 배치 단위로 삽입
        for i in range(0, total_records, batch_size):
            batch = data[i:i + batch_size]
            try:
                execute_batch(cur, insert_query, batch, page_size=batch_size)
                conn.commit()
                inserted_count += len(batch)
                print(f"삽입 진행중: {inserted_count}/{total_records} 레코드")
            except Exception as batch_error:
                print(f"배치 삽입 실패: {batch_error}")
                conn.rollback()
                
                # 배치 실패 시 개별 삽입 시도
                for record in batch:
                    try:
                        cur.execute(insert_query, record)
                        conn.commit()
                        inserted_count += 1
                    except Exception as record_error:
                        failed_count += 1
                        print(f"개별 레코드 삽입 실패: {record_error}")
                        conn.rollback()
        
        print(f"삽입 완료 - 성공: {inserted_count}개, 실패: {failed_count}개")
        
    except Exception as e:
        print(f"데이터 삽입 실패: {e}")
        raise
    finally:
        cur.close()
        conn.close()

try:
    print(f"CSV 파일 읽기 시작: {PG_UPLOAD_FILE_PATH}")
    
    encoding = 'utf-8'
    delimiter = ','

    # CSV 파일 읽기 (큰따옴표 처리 포함)
    df = pd.read_csv(
        PG_UPLOAD_FILE_PATH, 
        encoding=encoding, 
        delimiter=delimiter,
        quotechar='"',
        quoting=1  # QUOTE_MINIMAL
    )
    
    print(f"총 {len(df)}개 레코드 읽기 완료")
    print(f"컬럼 목록: {df.columns.tolist()}")
    
    # 데이터 준비
    prepared_data = prepare_data_from_csv(df)
    
    if prepared_data:
        # psycopg2를 사용한 데이터 삽입
        insert_data_with_psycopg2(prepared_data)
    else:
        print("삽입할 데이터가 없습니다")
    
except Exception as e:
    print(f"CSV 처리 실패: {e}")
    raise

CSV 파일 읽기 시작: /Users/toby/prog/kt/rubicon/data/sr_merged_product_202509231550.tsv
총 3496개 레코드 읽기 완료
컬럼 목록: ['disp_lv1', 'disp_lv2', 'disp_lv3', 'product_category_lv1', 'product_category_lv2', 'product_category_lv3', 'model_name', 'mdl_code', 'goods_id', 'goods_nm', 'color', 'release_date', 'aisc_yn', 'sc_yn', 'gc_yn', 'div_pay_apl_yn', 'show_yn', 'pd_url', 'dlgt_img_url', 'site_cd', 'created_at', 'usp_desc', 'review_num', 'estm_score', 'sale_prc1', 'sale_prc2', 'sale_prc3', 'review_content', 'disp_clsf_nm_list', 'spec', 'on_sale', 'web_cp_dc_amt', 'stock_qty', 'ctg_rank_recommend', 'ctg_rank_quantity', 'ctg_rank_rating', 'cstrt_mdl_codes', 'card_promotion', 'sale_prc']
총 3496개 레코드 준비 완료
삽입 진행중: 1000/3496 레코드
삽입 진행중: 2000/3496 레코드
삽입 진행중: 3000/3496 레코드
삽입 진행중: 3496/3496 레코드
삽입 완료 - 성공: 3496개, 실패: 0개


'/Users/toby/prog/kt/rubicon/data/sr_merged_product_202509231550.tsv'